# Model monitoring dashboard example

This notebook currently requires the plotly library to be installed.


In [ ]:
import pandas as pd
import weave
# Weave package now defaults to eager mode, but lazy mode required for this example notebook for now.
weave.use_lazy_execution()
from weave.legacy.scripts import syndata_mon
from weave.legacy.weave.arrow.list_ import dataframe_to_arrow
from weave.legacy.weave.ecosystem import wandb

## Create synthetic data


In [ ]:
data = syndata_mon.random_predictions(10)
timestamps = data._arrow_data.field('timestamp')
start_date = timestamps[0].as_py().to_pydatetime()
end_date = timestamps[-1].as_py().to_pydatetime()
predictions = weave.save(data, 'predictions')

## Create a Weave Board for the data


In [ ]:
# Create a dashboard for our data
board = weave.legacy.weave.panels.Board(
    vars={
        'x_max': weave.legacy.weave.ops.date_parse(str(end_date)),
        'x_min': lambda x_max: x_max - weave.legacy.weave.ops.days(5),
        'predictions_query': weave.legacy.weave.panels.Query(
            predictions,
            conditions=[
                lambda query_input: weave.legacy.weave.panels.QueryCondition(
                    expression=query_input['model_version'],
                    editor=weave.legacy.weave.panels.SelectEditor(
                        choices=query_input['model_version']
                    )
                ),
                lambda query_input: weave.legacy.weave.panels.QueryCondition(
                    expression=query_input['username'],
                    editor=weave.legacy.weave.panels.SelectEditor(
                        choices=query_input['username']
                    )
                ),
            ]),
        'predictions': lambda predictions_query: predictions_query.selected()
    },
    panels=[
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, x_min, x_max: wandb.TimeSeries(
                            predictions,
                            x=lambda item: item["timestamp"],
                            label=lambda item: item["model_version"],
                            agg=lambda preds: preds.count(),
                            min_x=x_min,
                            max_x=x_max,
                            mark="bar",
                            axis_labels={"y": "num_preds"},
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=0, w=12, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, x_min, x_max: wandb.TimeSeries(
                            predictions,
                            x=lambda item: item["timestamp"],
                            label=lambda item: item["username"],
                            agg=lambda preds: preds.count(),
                            min_x=x_min,
                            max_x=x_max,
                            mark="bar",
                            axis_labels={"y": "num_preds"},
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=12, y=0, w=12, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, x_min, x_max: wandb.TimeSeries(
                            predictions,
                            x=lambda item: item["timestamp"],
                            #label=lambda item: item["model_version"],
                            agg=lambda preds: preds["api_cost"].sum(),
                            min_x=x_min,
                            max_x=x_max,
                            mark="line",
                            axis_labels={"y": "api_cost"},
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=6, w=8, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, x_min, x_max: wandb.TimeSeries(
                            predictions,
                            x=lambda item: item["timestamp"],
                            #label=lambda item: item["model_version"],
                            agg=lambda preds: preds["prompt_tokens"].sum() + preds['completion_tokens'].sum(),
                            min_x=x_min,
                            max_x=x_max,
                            mark="line",
                            axis_labels={"y": "total_tokens"},
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=8, y=6, w=8, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, x_min, x_max: wandb.TimeSeries(
                            predictions,
                            x=lambda item: item["timestamp"],
                            #label=lambda item: item["model_version"],
                            agg=lambda preds: preds["latency"].avg(),
                            min_x=x_min,
                            max_x=x_max,
                            mark="line",
                            axis_labels={"y": "avg_latency"},
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=16, y=6, w=8, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, x_min, x_max: wandb.Distribution(
                predictions,
                value_fn=lambda pred: pred['api_cost'],
                bin_size=0.2
            ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=12, w=8, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, x_min, x_max: wandb.Distribution(
                predictions,
                value_fn=lambda pred: pred['prompt_tokens'] + pred['completion_tokens'],
                bin_size=25
            ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=8, y=12, w=8, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, x_min, x_max: wandb.Distribution(
                predictions,
                value_fn=lambda pred: pred['latency'],
                bin_size=0.05
            ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=16, y=12, w=8, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions: predictions,
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=18, w=24, h=12)
        )
    ]
)

In [ ]:
board